In [72]:
import numpy as np
import pandas as pd

In [73]:
#Opens the 'VEGPARM.TBL' file, reads its content, and splits it into lines
with open('VEGPARM.TBL') as fp:
  table_as_list = fp.read().splitlines()

#The for loop looks for the tables in the file and stores their names, sizes, and locations in a dictionary called 'tables'
tables = {}
for i, line in enumerate(table_as_list):
  if line == 'Vegetation Parameters':
    table_name = table_as_list[i+1]
    table_size = int(table_as_list[i+2].split(',')[0])
    tables[table_name] = dict(size = table_size, start = i+2, end = i+2+table_size)
    
tables

{'USGS': {'size': 27, 'start': 2, 'end': 29},
 'MODIFIED_IGBP_MODIS_NOAH': {'size': 20, 'start': 68, 'end': 88},
 'NLCD40': {'size': 40, 'start': 127, 'end': 167},
 'USGS-RUC': {'size': 28, 'start': 206, 'end': 234},
 'MODI-RUC': {'size': 21, 'start': 253, 'end': 274}}

In [74]:
#The function returns the values of "size", "start" and "end" of the called table
def obtain_table_values(tables, tabla):
    if tabla in tables:
        valores = tables[tabla]
        size = valores['size']
        start = valores['start']
        end = valores['end']
        return size, start, end
    else:
        return None  #Return None if the table do not exist
obtain_table_values(tables,'USGS')

(27, 2, 29)

In [75]:
#The function reads line by line and finds the name of the table, then return the data below in a str list
def read_tables(tables, tabla):
    datos_tabla = []
    if tabla in tables:
        for i, line in enumerate(table_as_list):
            if line == tabla:
                datos_tabla = table_as_list[obtain_table_values(tables,tabla)[1]:int(obtain_table_values(tables,tabla)[2]+1)]  # 1 for "start", 2 for "end"
        return datos_tabla
    else:
        return None  #Return None if the table do not exist
read_tables(tables,'USGS')

["27,1, 'SHDFAC NROOT   RS      RGL      HS      SNUP  MAXALB   LAIMIN  LAIMAX  EMISSMIN EMISSMAX ALBEDOMIN ALBEDOMAX  Z0MIN    Z0MAX    ZTOPV    ZBOTV'",
 "1,      .10,   1,    200.,   999.,   999.0,   0.04,    46.,    1.00,   1.00,   .880,    .880,     .15,      .15,      .50,     .50,     0.00,    0.00,     'Urban and Built-Up Land'  ",
 "2,      .80,   3,     40.,   100.,   36.25,   0.04,    66.,    1.56,   5.68,   .920,    .985,     .17,      .23,      .05,     .15,     0.50,    0.01,     'Dryland Cropland and Pasture' ",
 "3,      .80,   3,     40.,   100.,   36.25,   0.04,    66.,    1.56,   5.68,   .930,    .985,     .20,      .25,      .02,     .10,     0.50,    0.01,     'Irrigated Cropland and Pasture' ",
 "4,      .80,   3,     40.,   100.,   36.25,   0.04,    66.,    1.00,   4.50,   .920,    .985,     .18,      .23,      .05,     .15,     0.50,    0.01,     'Mixed Dryland/Irrigated Cropland and Pasture' ",
 "5,      .80,   3,     40.,   100.,   36.25,   0.04,    68.,    2.

In [129]:
#The function creates a DataFrame from the list returned from the function "read_table" and set the index and columns names
def tables_to_df(tables, tabla):
    if tabla in tables:
        df = pd.DataFrame(read_tables(tables, tabla))
        df.rename(columns={0:'col'},inplace=True)
        df = df.col.str.split(",", expand=True)
        df = df.drop(0, axis = 1)
        nombres = df.iloc[0,1]
        nombres = nombres.split()
        nombres.append("j")
        df = df.set_index(df.iloc[:, -1])
        df = df[1:]
        df.columns = nombres
        df.index.name = None
        df = df.drop("j", axis = 1)
        return df
    else:
        return None  #Return None if the table do not exist
    
tables_to_df(tables,'USGS')

,'SHDFAC,NROOT,RS,RGL,HS,SNUP,MAXALB,LAIMIN,LAIMAX,EMISSMIN,EMISSMAX,ALBEDOMIN,ALBEDOMAX,Z0MIN,Z0MAX,ZTOPV,ZBOTV'
'Urban and Built-Up Land',.10,1,200.,999.,999.0,0.04,46.,1.00,1.00,.880,.880,.15,.15,.50,.50,0.00,0.00
'Dryland Cropland and Pasture',.80,3,40.,100.,36.25,0.04,66.,1.56,5.68,.920,.985,.17,.23,.05,.15,0.50,0.01
'Irrigated Cropland and Pasture',.80,3,40.,100.,36.25,0.04,66.,1.56,5.68,.930,.985,.20,.25,.02,.10,0.50,0.01
'Mixed Dryland/Irrigated Cropland and Pasture',.80,3,40.,100.,36.25,0.04,66.,1.00,4.50,.920,.985,.18,.23,.05,.15,0.50,0.01
'Cropland/Grassland Mosaic',.80,3,40.,100.,36.25,0.04,68.,2.29,4.29,.920,.980,.18,.23,.05,.14,0.50,0.01
'Cropland/Woodland Mosaic',.80,3,70.,65.,44.14,0.04,60.,2.00,4.00,.930,.985,.16,.20,.20,.20,0.50,0.01
'Grassland',.80,3,40.,100.,36.35,0.04,70.,0.52,2.90,.920,.960,.19,.23,.10,.12,0.50,0.01
'Shrubland',.70,3,300.,100.,42.00,0.03,60.,0.50,3.66,.930,.930,.25,.30,.01,.05,0.50,0.10
'Mixed Shrubland/Grassland',.70,3,170.,100.,39.18,0.035,65.,0.60,2.60,.930,.950,.22,.30,.01,.06,0.50,0.10
'Savanna',.50,3,70.,65.,54.53,0.04,50.,0.50,3.66,.920,.920,.20,.20,.15,.15,5.00,0.10
